# Threading

In [79]:
# Synchronous

import requests
import time
import concurrent.futures
import threading

def download_site(url):
    requests.get(url)
    indicator = "J" if "jython" in url else "R"
    print(indicator, sep='', end='', flush=True)

def download_site_verbose(url):
    with requests.Session() as session:
        response = session.get(url)
        indicator = "J" if "jython" in url else "R"
        print(indicator, sep='', end='', flush=True)

sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80

start = time.perf_counter()
for url in sites:
    download_site(url)
duration = time.perf_counter() - start
print(f"Downloaded {len(sites)} sites in {duration} seconds")

JRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRJRDownloaded 160 sites in 7.431119749999198 seconds


In [81]:
start = time.perf_counter()
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    executor.map(download_site, sites)
duration = time.perf_counter() - start
print(f"Downloaded {len(sites)} sites in {duration} seconds")

RRJJJJJRRRJJRRJRRRJJJRRJJRRRJJJRRJJRJJJRRJRRRJJJRJRRJJRRJJRRJRJJRRRJRJJRRJJRJJRRJRRJJRRJJJRRJJRRJJRRJJRJRRJJRRJRRJJRRRJJJJJRRRJRJRJRRRJJJJRRJRJRJRJRJRJJRRRJJRJRDownloaded 160 sites in 1.4381094589989516 seconds


In [83]:
start = time.perf_counter()
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    executor.map(download_site_verbose, sites)
duration = time.perf_counter() - start
print(f"Downloaded {len(sites)} sites in {duration} seconds")

RRJJJJJRRRJRRJJJRJRRJRJJRRJJRRJRJJRRJRJRRJJRJRRJJRJJRJRRJJRRRJJJRRRRJJJRRJJRRJJRJRRJJRRRJJJRRJJRJJRRJJJRRRJRRJJRJRRJJRJJRRJRJRRJRJJJRJRRRJJRJRJRJRJJJRRRJRJJRJRRDownloaded 160 sites in 1.4759474589955062 seconds


In [87]:
# Locking, submit
class Account:
    def __init__(self):
        self.balance = 100 # shared data
        self.lock = threading.Lock()
    def update(self, transaction, amount):
        print(f'{transaction} thread updating...')
        with self.lock:
            local_copy = self.balance
            local_copy += amount
            time.sleep(1)
            self.balance = local_copy
        print(f'{transaction} thread finishing...')

account = Account()
print(f'starting with balance of {account.balance}')
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as ex:
    for transaction, amount in [('deposit', 50), ('withdrawal', -150)]:
        ex.submit(account.update, transaction, amount)
print(f'ending balance of {account.balance}')

starting with balance of 100
deposit thread updating...
withdrawal thread updating...
deposit thread finishing...
withdrawal thread finishing...
ending balance of 0


In [103]:
# Event
event = threading.Event()
print(event.is_set())
event.set()
print(event.is_set())
event.clear()
print(event.is_set())

False
True
False


In [107]:
s = threading.Semaphore(value=10)
s.acquire()
print(s._value)
s.release()
print(s._value)

9
10


In [119]:
import concurrent.futures
import random
import threading
import time

def welcome(semaphore, stop):
    while True and not stop.is_set():
        visitor_number = 0
        print(f'welcome visitor #{visitor_number}')
        semaphore.acquire() # reduces value, is blocked when the counter is zero until release is called
        visitor_number += 1
        time.sleep(random.random())
    
def monitor(semaphore, stop):
    while True and not stop.is_set():
        print(f'[monitor] semaphore={semaphore._value}')
        time.sleep(3)
        if semaphore._value == 0:
            print('[monitor] reached max users!')
            print('[monitor] kicking a user out...')
            semaphore.release() # increases value
            time.sleep(0.05)

stop = threading.Event()
semaphore = threading.Semaphore(value=10)
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    executor.submit(welcome, semaphore, stop)
    executor.submit(monitor, semaphore, stop)
    time.sleep(5)
    stop.set()

# Counting is atomic. This means that there is a guarantee that the operating system will not swap out the thread in the middle of incrementing or decrementing the counter.
# If a thread calls .acquire() when the counter is zero, that thread will block until a different thread calls .release() and increments the counter to one.

welcome visitor #0
[monitor] semaphore=9
welcome visitor #0
[monitor] semaphore=1
welcome visitor #0
welcome visitor #0
welcome visitor #0
welcome visitor #0
welcome visitor #0
[monitor] semaphore=3
welcome visitor #0
welcome visitor #0
[monitor] semaphore=1
welcome visitor #0
welcome visitor #0


KeyboardInterrupt: 

# MultiProcessing

In [85]:
import multiprocessing

start = time.perf_counter()
with multiprocessing.Pool() as pool:
    pool.map(download_site, sites)
duration = time.perf_counter() - start
print(f"Downloaded {len(sites)} sites in {duration} seconds")

# multiprocessing.Pool(initializer=set_global_session)
# If initializer is not None then each worker process will call initializer(*initargs) when it starts.

In [33]:
# Synchronous
import time

def calculate(limit):
    return sum(i * i for i in range(limit))

numbers = [5_000_000 + x for x in range(20)]
start = time.perf_counter()
for number in numbers:
    calculate(number)
duration = time.perf_counter() - start
print(f"Duration {duration} seconds")

Duration 4.790480667004886 seconds


In [39]:
# Concurrent
print(multiprocessing.cpu_count())
start = time.perf_counter()
with multiprocessing.Pool() as pool:
    pool.map(calculate, numbers)
duration = time.perf_counter() - start
print(f"Duration {duration} seconds")

# Asyncio

## Basics

## aiohttp

In [89]:
import asyncio
import time
import aiohttp

async def download_site(session, url):
    async with session.get(url) as response:
        indicator = "J" if "jython" in url else "R"
        print(indicator, sep='', end='', flush=True)

async def download_all_sites(sites):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in sites:
            task = asyncio.ensure_future(download_site(session, url))
            tasks.append(task)

        await asyncio.gather(*tasks, return_exceptions=True)

print("Starting downloads")
start = time.perf_counter()
loop = asyncio.get_event_loop()
loop.run_until_complete(download_all_sites(sites))
duration = time.perf_counter() - start
print(f"\nDownloaded {len(sites)} sites in {duration} seconds")

Starting downloads


RuntimeError: This event loop is already running

## aiofiles